# 2. Создание нейронной сети без использования готовых решений

__Автор__: Никита Владимирович Блохин (NVBlokhin@fa.ru)

Финансовый университет, 2020 г. 

In [373]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## 1. Создание нейронов и полносвязных слоев

1.1. Используя операции над матрицами и векторами из библиотеки `torch`, реализовать нейрон с заданными весами `weights` и `bias`. Прогнать вектор `inputs` через нейрон и вывести результат. 

In [374]:
class Neuron:

    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def forward(self, inputs):
        return torch.matmul(self.weights, inputs) + self.bias


In [375]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])
weights = torch.tensor([-0.2, 0.3, -0.5, 0.7])
bias = torch.tensor(3.14)

In [376]:
neuron = Neuron(weights, bias)
print(f'Neuron.forward: {neuron.forward(inputs)}')
print(f'functional.linear: {F.linear(inputs, weights, bias)}')

Neuron.forward: 4.840000152587891
functional.linear: 4.840000152587891


1.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать полносвязный слой с заданными весами `weights` и `biases`. Прогнать вектор `inputs` через слой и вывести результат. 

In [377]:
class Linear:

    def __init__(self, weights, biases):
        self.weights = weights
        self.biases = biases

    def forward(self, inputs):
        return torch.matmul(inputs, self.weights.T) + self.biases

In [378]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])
weights = torch.tensor([[-0.2, 0.3, -0.5, 0.7],
                        [0.5, -0.91, 0.26, -0.5],
                        [-0.26, -0.27, 0.17, 0.87]])  # .T - здесь точно Транспонирование нужно?

biases = torch.tensor([3.14, 2.71, 7.2])

In [379]:
m = Linear(weights, biases)
print(f'Linear.forward:\n{m.forward(inputs)}')
print(f'\nfunctional.linear:\n{F.linear(inputs, weights, biases)}')

Linear.forward:
tensor([ 4.8400,  0.1700, 10.3900])

functional.linear:
tensor([ 4.8400,  0.1700, 10.3900])


1.3 Реализовать полносвязный слой из __1.2__ таким образом, чтобы он мог принимать на вход матрицу (батч) с данными. Продемонстрировать работу.
Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.


In [380]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

In [381]:
m = Linear(weights, biases)
print(f'Linear.forward:\n{m.forward(inputs)}')
print(f'\nfunctional.linear:\n{F.linear(inputs, weights, biases)}')

Linear.forward:
tensor([[ 3.7900,  0.9200,  9.0850],
        [ 6.1400, -2.1000,  6.9000],
        [ 2.0400,  0.7610,  6.7260]])

functional.linear:
tensor([[ 3.7900,  0.9200,  9.0850],
        [ 6.1400, -2.1000,  6.9000],
        [ 2.0400,  0.7610,  6.7260]])


1.4 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения). Прогнать вектор `inputs` через слой и вывести результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

In [382]:
class Linear:

    def __init__(self, n_features, n_neurons):
        self.n_features = n_features
        self.n_neurons = n_neurons

        self.weights = torch.randn(n_neurons, n_features)
        self.biases = torch.randn(n_neurons)

    def forward(self, inputs):
        return torch.matmul(inputs, self.weights.T) + self.biases

In [383]:
torch.manual_seed(0)

m = Linear(4, 3)
m.forward(inputs)

tensor([[ -5.0178,   0.5240,  -3.9319],
        [  4.0738,  -6.7887,  -3.5657],
        [-11.6052,  -0.3882,  -3.1959]])

In [384]:
# проверка
nn_m = nn.Linear(4, 3)
nn_m.weight = nn.Parameter(m.weights)
nn_m.bias = nn.Parameter(m.biases)
nn_m.forward(inputs)

tensor([[ -5.0178,   0.5240,  -3.9319],
        [  4.0738,  -6.7887,  -3.5657],
        [-11.6052,  -0.3882,  -3.1959]], grad_fn=<AddmmBackward0>)

1.5 Используя решение из __1.4__, создать 2 полносвязных слоя и пропустить матрицу `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выбрать произвольно, количество нейронов во втором слое выбрать так, чтобы результатом прогона являлась матрица (3x7). 

In [385]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

In [386]:
class NeuralNetwork:

    def __init__(self):
        self.h1 = Linear(4, 3)
        self.o1 = Linear(3, 7)

    def forward(self, inputs):
        return self.o1.forward(self.h1.forward(inputs))


In [387]:
torch.manual_seed(0)

m = NeuralNetwork()
m.forward(inputs)

tensor([[  5.1553,   1.1916,  -4.7817,   3.6611,  -9.6546,   0.1692,   5.8099],
        [ -0.8347,   0.0913,   3.1227,   3.5542,   5.4400,   4.9157,   8.2533],
        [  8.8057,  -1.1342,  -7.8488,   6.7415, -11.1044,   5.1500,   5.6152]])